In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# ⚡ Define Ticker Symbol (Change this for different stocks)
TICKER = "AAPL"

# ⚡ Fetch Stock Data
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1d", interval="1m")  # 1-day, minute-by-minute
    if hist.empty:
        raise ValueError(f"No stock data found for {ticker}")
    
    latest_price = hist["Close"].iloc[-1]  # Get last available stock price
    return torch.tensor(latest_price, dtype=torch.float32)  # Convert directly to torch.Tensor

# ⚡ Fetch Option Chain Data
def fetch_option_data(ticker):
    stock = yf.Ticker(ticker)
    expiration_dates = stock.options  # List of expiration dates
    if not expiration_dates:
        raise ValueError(f"No options data found for {ticker}")
    
    # Select the nearest expiration date
    expiry = expiration_dates[0]
    opt_chain = stock.option_chain(expiry)

    # Merge calls & puts
    options = pd.concat([opt_chain.calls, opt_chain.puts])

    # Convert relevant columns to PyTorch tensors
    features = ["strike", "impliedVolatility", "delta", "gamma", "theta", "vega", "rho"]
    options = options[features].dropna()

    # Convert pandas dataframe to torch tensor
    option_tensor = torch.tensor(options.values, dtype=torch.float32)

    return option_tensor

# ⚡ Combine Stock & Option Data into GAN Input Format
def create_market_data(ticker):
    stock_price = fetch_stock_data(ticker)
    options = fetch_option_data(ticker)
    
    # Add stock price as a column (broadcasting)
    stock_prices = stock_price.expand(options.shape[0], 1)  # Expand to match batch size

    # Concatenate stock price with option data
    market_data = torch.cat([stock_prices, options], dim=1)

    # Save as a PyTorch file instead of CSV (better for deep learning)
    torch.save(market_data, "market_data.pt")
    print("✅ Market data saved as market_data.pt (PyTorch format)")

    return market_data

# ⚡ Fetch & Save Data
market_data = create_market_data(TICKER)
print(market_data.shape)  # Should match (num_options, 8)


KeyError: "['delta', 'gamma', 'theta', 'vega', 'rho'] not in index"

In [10]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim=4):  # 4 outputs for multi-leg options
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()  # Outputs between -1 and 1 (scaled trading positions)
        )
    
    def forward(self, x):
        trades = self.model(x)
        return self.apply_strategy_constraints(trades, x)
    
    def apply_strategy_constraints(self, trades, market_state):
        """
        Apply constraints to ensure valid multi-leg strategies (Iron Condors, Straddles, Spreads)
        market_state contains: stock price, IV, portfolio Greeks, etc.
        """
        stock_price = market_state[:, 0]  # Assuming first feature is stock price
        
        # Generate valid strikes based on stock price
        atm_strike = stock_price.round()  # Closest ATM strike
        strikes = torch.stack([
        atm_strike - 5, 
        atm_strike, 
        atm_strike + 5, 
        atm_strike + 10
        ], dim=1).to(trades.device)

        # Ensure strikes follow valid order for an iron condor
        trades[:, 0] = -torch.abs(trades[:, 0])  # Sell lower call
        trades[:, 1] = torch.abs(trades[:, 1])   # Buy higher call
        trades[:, 2] = -torch.abs(trades[:, 2])  # Sell lower put
        trades[:, 3] = torch.abs(trades[:, 3])   # Buy higher put
        
        return trades


In [11]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),  # Output a single scalar for each strategy
            nn.Sigmoid()  # Probability of being a valid trade (0 to 1)
        )
    
    def forward(self, x):
        return self.model(x)

    def evaluate_trade(self, trades, market_state):
        """
        Custom function to check if generated strategies are valid & risk-aware
        """
        # Check if iron condor or straddle is correctly structured
        strategy_validity = ((trades[:, 0] < trades[:, 1]) & (trades[:, 2] < trades[:, 3])).float()

        # Penalize excessive delta exposure (violates delta-neutrality)
        delta_violation = torch.abs(market_state[:, -4])  # Portfolio delta
        risk_penalty = torch.clamp(delta_violation - 0.05, min=0)  # Excess delta exposure

        # Final validity score
        validity = self.forward(trades).squeeze(1)  # Flatten to a 1D tensor of shape [batch_size]
        return validity - risk_penalty


In [12]:
# Define model input dimensions
input_dim = 10  # Features: stock prices, IV, portfolio Greeks, etc.
output_dim = 4  # Number of strategy legs (for multi-leg options)

# Initialize models
G = Generator(input_dim, output_dim).to(device)  # Generator
D = Discriminator(output_dim).to(device)  # Discriminator

# Optimizers
g_optimizer = optim.Adam(G.parameters(), lr=0.001)
d_optimizer = optim.Adam(D.parameters(), lr=0.001)


In [13]:
# Define training parameters
num_epochs = 1000  
batch_size = 32  # Number of trades per batch
# Define loss function
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for GAN training

In [ ]:
for epoch in range(num_epochs):
    noise = torch.randn(32, input_dim).to(device)
    market_state = torch.randn(32, input_dim).to(device)  # Simulated market data

    # Generate structured multi-leg strategies
    fake_trades = G(market_state)

    # Real & Fake labels
    real_labels = torch.ones(batch_size).to(device)  # Real labels (shape: [32])
    fake_labels = torch.zeros(batch_size).to(device)  # Fake labels (shape: [32])

    # Generate structured multi-leg strategies
    fake_trades = G(market_state)
    # Example: Load actual market trades from a dataset
    real_trades = load_real_trades_data(batch_size).to(device)

    # Train Discriminator
    d_optimizer.zero_grad()

    real_scores = D(real_trades)  # Real trades (discriminator outputs shape: [32])
    fake_scores = D(fake_trades)  # Fake trades (discriminator outputs shape: [32])

    d_loss = criterion(real_scores, real_labels) + criterion(fake_scores, fake_labels)
    d_loss.backward()
    d_optimizer.step()

    # Train Generator
    g_optimizer.zero_grad()
    fake_scores = D(fake_trades)
    g_loss = criterion(fake_scores, real_labels)  # Generator wants to fool the discriminator
    g_loss.backward()
    g_optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")


NameError: name 'load_real_trades_data' is not defined

: 